# Import Libraries

In [1]:
import os
import sys
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from scipy.io import loadmat
import warnings
warnings.simplefilter("ignore")

# IMPORT PYOD PACKAGES AND METHODS

In [2]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

# IMPORT METRICES PACKAGES

In [3]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

# Define Data File and read X and Y

In [4]:
mat_file_list=["arrhythmia.mat","cardio.mat","glass.mat","ionosphere.mat","letter.mat","lympho.mat","mnist.mat","musk.mat",
              "optdigits.mat","pendigits.mat","pima.mat","satellite.mat","satimage-2.mat","shuttle.mat","vertebral.mat",
               "vowels.mat","wbc.mat"]
mat_file_list

['arrhythmia.mat',
 'cardio.mat',
 'glass.mat',
 'ionosphere.mat',
 'letter.mat',
 'lympho.mat',
 'mnist.mat',
 'musk.mat',
 'optdigits.mat',
 'pendigits.mat',
 'pima.mat',
 'satellite.mat',
 'satimage-2.mat',
 'shuttle.mat',
 'vertebral.mat',
 'vowels.mat',
 'wbc.mat']

# To load Mat file

In [5]:
data=loadmat("data/cardio.mat")
data

{'__header__': b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
         -0.28978574, -0.49329397],
        [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
         -0.25638541, -0.49329397],
        [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
         -0.25638541,  1.14001753],
        ...,
        [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
          0.24461959, -0.49329397],
        [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
          0.14441859, -0.49329397],
        [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
          3.58465295, -0.49329397]]),
 'y': array([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])}

In [6]:
len(data)

5

In [7]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [8]:
data.values()

dict_values([b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC', '1.0', [], array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
        -0.28978574, -0.49329397],
       [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
        -0.25638541, -0.49329397],
       [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
        -0.25638541,  1.14001753],
       ...,
       [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
         0.24461959, -0.49329397],
       [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
         0.14441859, -0.49329397],
       [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
         3.58465295, -0.49329397]]), array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]])])

# Input(Independent) feature shape in Mat file format

In [9]:
type(data["X"]),data["X"].shape

(numpy.ndarray, (1831, 21))

# Dependent/Target/Output feature shape 


In [10]:
type(data["y"]),data["y"].shape

(numpy.ndarray, (1831, 1))

In [11]:
df_columns=["Data","#Samples","# Dimensions","Outlier Perc","ABOD","CBLOF","FB","HBOS",
            "IForest","KNN","LOF","MCD","OCSVM","PCA"]


# ROC PERFORMANCE  EVOLUTION TABLE

In [12]:
roc_df=pd.DataFrame(columns=df_columns)  

In [13]:
roc_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


# Precision_n_Scores performance evolution table

In [14]:
prn_df=pd.DataFrame(columns=df_columns)
prn_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


# Time DataFrame

In [15]:
time_df=pd.DataFrame(columns=df_columns)
time_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


# Exploring all Mat files

In [16]:
from time import time
random_state=np.random.RandomState(42)
for mat_file in mat_file_list:
    print("\n...Processing ",mat_file,"...")
    mat=loadmat(os.path.join('data',mat_file))
    
    X=mat["X"]
    y=mat["y"].ravel() # converting 2D TO 1D
    
    outliers_fraction=np.count_nonzero(y)/len(y)
    
    outliers_percentage=round(outliers_fraction*100,ndigits=4)

    
    #Construct Containers for saving results
    
    roc_list=[mat_file[:-4],X.shape[0],X.shape[1],outliers_percentage]
    prn_list=[mat_file[:-4],X.shape[0],X.shape[1],outliers_percentage]
    time_list=[mat_file[:-4],X.shape[0],X.shape[1],outliers_percentage]
    
    
    #60% Data for training and 40% for testing
    
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.4,random_state=random_state)
    
    
    #Standardizing data for processing
    
    
    X_train_norm,X_test_norm=standardizer(X_train,X_test)
    
    classifiers={"Angle based  outlier detector (ABOD)" : ABOD(contamination=outliers_fraction),
                "Cluster based local outlier factor" : CBLOF(contamination=outliers_fraction,check_estimator=False,
                 random_state=random_state),
                 "Feature Bagging ":FeatureBagging (contamination=outliers_fraction,random_state=random_state),
                 "Histogram base outlier detector(HBOS)" :HBOS(contamination=outliers_fraction),
                 "Isolation Forest":IForest(contamination=outliers_fraction,random_state=random_state),
                 "K nearest neighbours(KNN)":KNN(contamination=outliers_fraction),
                 "Local outlier factor(LOF)":LOF(contamination=outliers_fraction),
                 "Minimum covariance determinant(MCD)":MCD(contamination=outliers_fraction,random_state=random_state),
                 "one class svm(OCSVM)":OCSVM(contamination=outliers_fraction),
                 "principal component analysis(PCA)":PCA(contamination=outliers_fraction,random_state=random_state)}
    
    for clf_name,clf in classifiers.items():
        t0=time()
        clf.fit(X_train_norm)
        test_scores=clf.decision_function(X_test_norm)
        t1=time()
        duration=round(t1 - t0, ndigits=4)
        time_list.append(duration)
        
        roc = round(roc_auc_score(y_test,test_scores),ndigits=4)
        prn = round(precision_n_scores(y_test,test_scores),ndigits=4)
        
        print("{clf_name} ROC:{roc},precision @ rank n:{prn},"
             "execution time:{duration}s".format(clf_name=clf_name,roc=roc,prn=prn,duration=duration))

        roc_list.append(roc)
        prn_list.append(prn)
        
    temp_df=pd.DataFrame(time_list).transpose()
    temp_df.columns=df_columns
    time_df=pd.concat([time_df,temp_df],axis=0)
    
    temp_df=pd.DataFrame(roc_list).transpose()
    temp_df.columns=df_columns
    roc_df=pd.concat([roc_df,temp_df],axis=0)
    
    temp_df=pd.DataFrame(prn_list).transpose()
    temp_df.columns=df_columns
    prn_df=pd.concat([prn_df,temp_df],axis=0)
    
    
    
    


...Processing  arrhythmia.mat ...
Angle based  outlier detector (ABOD) ROC:0.7687,precision @ rank n:0.3571,execution time:1.2811s
Cluster based local outlier factor ROC:0.7684,precision @ rank n:0.4643,execution time:1.4991s
Feature Bagging  ROC:0.7799,precision @ rank n:0.5,execution time:0.489s
Histogram base outlier detector(HBOS) ROC:0.8511,precision @ rank n:0.5714,execution time:1.6561s
Isolation Forest ROC:0.8527,precision @ rank n:0.5714,execution time:0.391s
K nearest neighbours(KNN) ROC:0.782,precision @ rank n:0.5,execution time:0.073s
Local outlier factor(LOF) ROC:0.7787,precision @ rank n:0.4643,execution time:0.059s
Minimum covariance determinant(MCD) ROC:0.8228,precision @ rank n:0.4286,execution time:0.8731s
one class svm(OCSVM) ROC:0.7986,precision @ rank n:0.5,execution time:0.031s
principal component analysis(PCA) ROC:0.7997,precision @ rank n:0.5,execution time:0.046s

...Processing  cardio.mat ...
Angle based  outlier detector (ABOD) ROC:0.5763,precision @ rank n

Cluster based local outlier factor ROC:0.9609,precision @ rank n:0.3077,execution time:0.276s
Feature Bagging  ROC:0.4687,precision @ rank n:0.0462,execution time:3.8243s
Histogram base outlier detector(HBOS) ROC:0.9294,precision @ rank n:0.2615,execution time:0.012s
Isolation Forest ROC:0.9422,precision @ rank n:0.2769,execution time:0.638s
K nearest neighbours(KNN) ROC:0.7602,precision @ rank n:0.0462,execution time:0.545s
Local outlier factor(LOF) ROC:0.481,precision @ rank n:0.0462,execution time:0.461s
Minimum covariance determinant(MCD) ROC:0.8271,precision @ rank n:0.0615,execution time:2.0491s
one class svm(OCSVM) ROC:0.93,precision @ rank n:0.2923,execution time:0.7471s
principal component analysis(PCA) ROC:0.9332,precision @ rank n:0.3385,execution time:0.007s

...Processing  pima.mat ...
Angle based  outlier detector (ABOD) ROC:0.6757,precision @ rank n:0.5106,execution time:0.137s
Cluster based local outlier factor ROC:0.684,precision @ rank n:0.4681,execution time:0.081s
F

In [17]:
roc_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.7687,0.7684,0.7799,0.8511,0.8527,0.782,0.7787,0.8228,0.7986,0.7997
0,cardio,1831,21,9.6122,0.5763,0.8221,0.4879,0.8453,0.9414,0.6959,0.4715,0.8778,0.9507,0.9638
0,glass,214,9,4.2056,0.7104,0.8506,0.7043,0.6524,0.7195,0.7805,0.7774,0.7165,0.6189,0.622
0,ionosphere,351,33,35.8974,0.9004,0.8952,0.8933,0.5195,0.8309,0.9134,0.8989,0.9399,0.8372,0.7971
0,letter,1600,32,6.25,0.8465,0.7423,0.866,0.5728,0.5778,0.845,0.8409,0.7499,0.5744,0.48
0,lympho,148,18,4.0541,0.9382,0.9709,0.9673,0.9964,0.9855,0.9636,0.9636,0.9164,0.9636,0.9818
0,mnist,7603,100,9.2069,0.7813,0.8447,0.7259,0.5675,0.7801,0.8409,0.7085,0.863,0.8417,0.8396
0,musk,3062,166,3.1679,0.0809,1,0.5228,0.9999,0.9996,0.7348,0.5323,1,1,1
0,optdigits,5216,64,2.8758,0.4428,0.7852,0.4641,0.8822,0.5764,0.3824,0.4584,0.3486,0.4972,0.504
0,pendigits,6870,16,2.2707,0.7008,0.9609,0.4687,0.9294,0.9422,0.7602,0.481,0.8271,0.93,0.9332


In [18]:
prn_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.3571,0.4643,0.5,0.5714,0.5714,0.5,0.4643,0.4286,0.5,0.5
0,cardio,1831,21,9.6122,0.1875,0.4844,0.1406,0.4688,0.5,0.2812,0.125,0.3906,0.5938,0.6875
0,glass,214,9,4.2056,0.25,0.25,0.25,0,0.25,0.25,0.25,0,0.25,0.25
0,ionosphere,351,33,35.8974,0.8214,0.8036,0.75,0.3393,0.6607,0.8393,0.75,0.8571,0.7143,0.5893
0,letter,1600,32,6.25,0.275,0.175,0.4,0.125,0.05,0.3,0.325,0.075,0.1,0.05
0,lympho,148,18,4.0541,0.4,0.6,0.6,0.8,0.6,0.6,0.6,0.6,0.6,0.8
0,mnist,7603,100,9.2069,0.3562,0.4007,0.3664,0.1199,0.2979,0.4144,0.339,0.3973,0.3801,0.3767
0,musk,3062,166,3.1679,0.0333,1,0.1667,0.9667,0.9333,0.2333,0.1333,0.9667,1,1
0,optdigits,5216,64,2.8758,0.0161,0,0.0484,0.2581,0.0161,0,0.0484,0,0,0
0,pendigits,6870,16,2.2707,0.0308,0.3077,0.0462,0.2615,0.2769,0.0462,0.0462,0.0615,0.2923,0.3385


In [19]:
time_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,1.2811,1.4991,0.489,1.6561,0.391,0.073,0.059,0.8731,0.031,0.046
0,cardio,1831,21,9.6122,0.404,0.149,0.7321,0.006,0.405,0.156,0.105,0.508,0.062,0.003
0,glass,214,9,4.2056,0.041,0.043,0.043,0.003,0.319,0.011,0.003,0.033,0.001,0.002
0,ionosphere,351,33,35.8974,0.077,0.057,0.07,0.009,0.312,0.016,0.006,0.055,0.005,0.003
0,letter,1600,32,6.25,0.381,0.114,0.6,0.014,0.406,0.126,0.085,0.9731,0.052,0.004
0,lympho,148,18,4.0541,0.032,0.039,0.029,0.005,0.286,0.007,0.002,0.034,0.002,0.001
0,mnist,7603,100,9.2069,6.3035,1.0421,40.5331,0.045,1.6251,5.4464,4.9584,2.8412,3.1293,0.107
0,musk,3062,166,3.1679,1.8611,0.362,10.0158,0.059,1.0701,1.4391,1.2851,13.633,0.8801,0.125
0,optdigits,5216,64,2.8758,2.6712,0.463,11.8659,0.031,0.8211,1.4051,1.2991,1.2211,1.0041,0.037
0,pendigits,6870,16,2.2707,1.5111,0.276,3.8243,0.012,0.638,0.545,0.461,2.0491,0.7471,0.007
